<a href="https://colab.research.google.com/github/nikitasaxena992/Machine_Learning/blob/main/Tweet_Topic_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import scipy
import pandas as pd
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
from statistics import mean
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
class SimilarityCalculator:
  def __init__(self, text, df):
    self.text = text
    self.df = df
    self.text_embedded = self.get_msg_embedded([text])
    self.keyword_embedded = self.get_embedded_dict(df["keyword"])
    self.topic_embedded = self.get_embedded_dict(df["topic"])

  def get_embedded_dict(self, data):
    dict_embedded = {}
    for words in data:
      word = words.split(",")
      for key in word:
        stripped_key = key.strip()
        if stripped_key not in dict_embedded:
          dict_embedded[stripped_key] = self.get_msg_embedded([stripped_key])
    return dict_embedded

  def get_msg_embedded(self, input):
    return model(input)

  def similarity_measure_with_text(self, word_embedded):
    distance = scipy.spatial.distance.cdist(self.text_embedded, word_embedded, "cosine")[0]
    return (1-distance)[0]

  def get_similarity_from_keyword(self, df):
    data_max = []
    data_avg = []
    for keywords in df['keyword']:
      similarity_measure_list=[]
      keyword = keywords.split(",")
      for word in keyword:
        stripped_keyword = word.strip()
        key_emb = self.keyword_embedded[stripped_keyword]
        similarity_measure_list.append(self.similarity_measure_with_text(key_emb))
      data_max.append(max(similarity_measure_list))
      data_avg.append(mean(similarity_measure_list))
    return data_max, data_avg

  def get_similarity_from_topic(self, df):
    data = []
    for topic in df['topic']:
      topic_emb = self.topic_embedded[topic]
      data.append(self.similarity_measure_with_text(topic_emb))
    return data

  @staticmethod
  def get_clean_text_keywords(text):
    # For tweets we will need to remove hyperlinks, @mentions, any punctuations and non alphanumeric characters.
    text = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)',' ',text)
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text.lower().split() if word not in set(stopwords.words('english'))]
    text = list(set(text))
    return text

  def get_similarity_of_topic_by_direct_matching(self, df):
    direct_matching_topic = []
    for topic in df['topic']:
      if topic in SimilarityCalculator.get_clean_text_keywords(self.text):
        direct_matching_topic.append(1)
      else:
        direct_matching_topic.append(0)
    
    return direct_matching_topic

    
  def get_similarity_of_keyword_by_direct_matching(self, df):
    direct_matching_keyword = []
    for keyword in df['keyword']:
      sum_matching_keys = 0
      for key in list(set(keyword.split(","))):
          if key.strip() in SimilarityCalculator.get_clean_text_keywords(self.text):
            sum_matching_keys += 1
      direct_matching_keyword.append(sum_matching_keys)
    return direct_matching_keyword


In [3]:
!pip install flask-restful 
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:

from flask import Flask, jsonify, request

from flask import Flask
from flask_ngrok import run_with_ngrok


!ngrok authtoken '2HSKbKu8Xk9hQ1H4BInJMBqgGqK_27gzegbTMUCqG3Wvj1ce1'

app = Flask(__name__)
run_with_ngrok(app)   

# on the terminal type: curl http://127.0.0.1:5000/
# returns hello world when we use GET.
# returns the data that we send when we use POST.
@app.route('/', methods = ['GET', 'POST'])
def home():
    if(request.method == 'GET'):
     df = pd.read_csv("/content/Topic Data.csv")
     text = ["In the Super 12 stage of this #T20WorldCup   , 99 bowlers were used. The highest Bowling Impact recorded was that of Anrich Nortje, reaching a vast +88. The second lowest was Kagiso Rabada, on -39.",
        "#BREAKING - Liquorgate Case: Manish Sisodia aide Dinesh Arora turns approver.#ManishSisodia",
        "For us this world cup is like 2011 ODI World Cup. For Pakistan it's like 2017 CT or 1992 WC. And for England and New Zealand it's like 2019 ODI world cup",
        "Countries with most Men's Player of the Month awards Five different players won it for India (Pant, Ashwin, Bhuvi, Shreyas, Kohli).",
        "PM Modi is now speaking for himself, not speaking for people as some outside force but speaking as one of the people who has been successful, that is very threatening to the intellectuals",
        "EXCLUSIVE: #RanveerSingh and #Shankar to team up on the biggest Pan Indian Cinematic event - #Shankar to direct #Ranveer in a 3-part film based on the iconic #Velpari Novel. Filming begins in 2023. Read more details",
        "UK Court approves extradition of Robert Vadra’s aide, Sanjay Bhandari, who is facing ED, CBI probe in PMLA case in India",
        "Gujarat elections: Can PM Modi bank on Gujarati pride again? Watch the panelists discuss on #TTP with ",
        "Massive support for the BJP across Gujarat. Watch from Kaprada.",
        "Dev Diwali is special and Dev Diwali in Kashi is even more memorable. Have a look at these magnificent pictures from the eternal city of Kashi…",
        "JUST IN - Elon Musk urges independent voters to cast ballots for Republicans in the U.S. midterm election."]

    text=["I like to roam around a lot, yesterday I went to Mountains. cruise"]
    text_df = pd.DataFrame(text, columns=['text'])
    text_df
    final_df = pd.DataFrame(columns=['text', "topic", "keyword", "topic_text_similarity", "keyword_max",  "keyword_avg", "direct_matching_keyword", "direct_matching_topic"])

    for text in text_df["text"]:
        sm_calculator = SimilarityCalculator(text, df)

        # Get topic text similarity
        topic_similarity = sm_calculator.get_similarity_from_topic(df)

        # Get max and average of keyword text similarity
        keyword_max, keyword_avg = sm_calculator.get_similarity_from_keyword(df)

        # Get similarity by direct matching of keywords
        direct_matching_keyword = sm_calculator.get_similarity_of_keyword_by_direct_matching(df)

        # Get similarity by direct matching of topics
        direct_matching_topic = sm_calculator.get_similarity_of_topic_by_direct_matching( df)

        #Assign all values to df
        df["text"] = text
        
        df["topic_text_similarity"] = topic_similarity
        df["keyword_max"] = keyword_max
        df["keyword_avg"] = keyword_avg
        df["direct_matching_keyword"] = direct_matching_keyword
        df["direct_matching_topic"] = direct_matching_topic
        final_df = final_df.append(df, ignore_index = True)

        final_df= final_df.sort_values('topic_text_similarity',ascending=False)


    def word_avg(topic_text_simi,keywrd_max,keywrd_avg,direct_match_keyword,direct_match_topic):
        topic_text_simi = 30* topic_text_simi
        keywrd_max = 30 * keywrd_max
        keywrd_avg = 20* keywrd_avg

        if direct_match_keyword > 1:
            direct_match_keyword =  10 * 1.5

        else:
            direct_match_keyword = direct_match_keyword* 10

        if direct_match_topic > 1:
            direct_match_topic = 10 * 1.5
        else:
            direct_match_topic =  10 * direct_match_topic

        return topic_text_simi + keywrd_max + keywrd_avg + direct_match_keyword + direct_match_topic

        

    final_df['avaerage_score'] = final_df.apply(lambda x: word_avg(x['topic_text_similarity'],x['keyword_max'],x['keyword_avg'],x['direct_matching_keyword'],x['direct_matching_topic']),axis=1)

    return final_df.to_dict()

app.run()


/bin/bash: ngrok: command not found
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4f38-104-196-137-191.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Nov/2022 17:25:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2022 17:25:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2022 17:25:26] "GET / HTTP/1.1" 200 -
